In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
import random
import numpy as np
from konlpy.tag import Mecab;tagger=Mecab()
from collections import Counter
%matplotlib inline  

* https://arxiv.org/pdf/1412.6581.pdf
* https://arxiv.org/pdf/1511.06349.pdf

In [2]:
data = open('../../DOMAIN_10D_300EA_DATA_170427.txt','r',encoding='utf-8').readlines()

In [3]:
data = [d.split('\t')[0] for d in data if 'FLOWER' in d.split('\t')[1]]

In [4]:
BATCH_SIZE = len(data)

In [5]:
SEQ_LENGTH=15
SOS_token = 0
EOS_token = 1

In [6]:
train=[]

In [7]:
for t0 in data:
    t0 = t0.replace("<br>","")
    t0 = t0.replace("/","")
    
    token0 = tagger.morphs(t0)
    
    if len(token0)>=SEQ_LENGTH:
        token0= token0[:SEQ_LENGTH-1]
    token0.append("EOS")

    while len(token0)<SEQ_LENGTH:
        token0.append('PAD')
    
    train.append([token0,token0])

In [8]:
n_words=4
word2index={"SOS":0,"EOS":1,"PAD":2,"UNK":3}

for t in train:
    for token in t[0]:
        if token not in word2index:
            word2index[token]=n_words
            n_words+=1

index2word = {v:k for k,v in word2index.items()}

In [9]:
def remove_list(x):
    del x[:]
    del x

In [10]:
def prepare_sequence(seq, to_ix):
    idxs = list(map(lambda w: to_ix[w], seq))
    tensor = torch.LongTensor(idxs)
    return Variable(tensor)

In [11]:
train_x=[]
train_y=[]
lengths=[]
for tr in train:
    temp = prepare_sequence(tr[0], word2index)
    temp = temp.view(1,-1)
    train_x.append(temp)

    temp2 = prepare_sequence(tr[1],word2index)
    temp2 = temp2.view(1,-1)
    train_y.append(temp2)
    
    length = [t for t in tr[1] if t !='PAD']
    lengths.append(len(length))

inputs = torch.cat(train_x)
targets = torch.cat(train_y)

remove_list(train_x)
remove_list(train_y)

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size,latent_size=10,n_layers=1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.Wmu= nn.Linear(hidden_size,latent_size)
        self.Wsigma = nn.Linear(hidden_size,latent_size)
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,batch_first=True)
    
    def reparametrize(self, mu, log_var):
        """"z = mean + eps * sigma where eps is sampled from N(0, 1)."""
        eps = Variable(torch.randn(mu.size(0), mu.size(1)))
        z = mu + eps * torch.exp(log_var/2)    # 2 for convert var to std
        return z
    
    def forward(self, input,train=True):
        hidden = Variable(torch.zeros(self.n_layers, input.size(0), self.hidden_size)) 
        
        embedded = self.embedding(input)
        output, hidden = self.gru(embedded, hidden)
        mu = self.Wmu(hidden[-1])
        log_var = self.Wsigma(hidden[-1])
        z = self.reparametrize(mu, log_var)
        
        return z,mu,log_var

In [13]:
encoder_test = EncoderRNN(len(word2index), 100,10,2)
print(encoder_test)

EncoderRNN (
  (Wmu): Linear (100 -> 10)
  (Wsigma): Linear (100 -> 10)
  (embedding): Embedding(780, 100)
  (gru): GRU(100, 100, num_layers=2, batch_first=True)
)


In [14]:
out, mu,log_var = encoder_test(inputs[:3].view(3,-1))

In [15]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size,latent_size=10, n_layers=1):
        super(DecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.Wz = nn.Linear(latent_size,hidden_size)
        self.tanh = nn.Tanh()
        
        # Define the layers
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)

        #self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size, self.n_layers,batch_first=True)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input,latent,lengths,seq_length,training=True):
        
        # Get the embedding of the current input word
        embedded = self.embedding(input)
        #embedded = self.dropout(embedded)
        
        # h0
        if training:
            hidden = self.tanh(self.Wz(latent)).view(self.n_layers,BATCH_SIZE,-1) 
        else:
            hidden = self.tanh(self.Wz(latent)).view(self.n_layers,1,-1)

        decode=[]
        # Apply GRU to the output so far
        for i in range(seq_length):
            
            _, hidden = self.gru(embedded, hidden)
            score = self.out(hidden.view(hidden.size(0)*hidden.size(1),-1))
            softmaxed = F.log_softmax(score)
            decode.append(softmaxed)
            _,input = torch.max(softmaxed,1)
            embedded = self.embedding(input)
            #embedded = self.dropout(embedded)
        
        # 요고 주의! time-step을 column-wise concat한 후, reshape!!
        scores = torch.cat(decode,1)
        remove_list(decode)
        
        return scores.view(input.size(0)*seq_length,-1)

In [16]:
HIDDEN_SIZE = 50
LATENT_SIZE = 10
STEP=3000
LEARNING_RATE=0.001

In [17]:
encoder =  EncoderRNN(len(word2index), HIDDEN_SIZE,LATENT_SIZE, 2)
decoder = DecoderRNN(HIDDEN_SIZE,len(word2index),LATENT_SIZE)
Recon = nn.CrossEntropyLoss()
enc_optim= torch.optim.Adam(encoder.parameters(), lr=LEARNING_RATE)
dec_optim = torch.optim.Adam(decoder.parameters(),lr=LEARNING_RATE)

In [66]:
for epoch in range(STEP):
    
    #KCA = 0.3
    encoder.zero_grad()
    decoder.zero_grad()
    
    decoder_input = Variable(torch.LongTensor([[SOS_token]*BATCH_SIZE])).transpose(1,0)
    normal = Variable(torch.randn([1,BATCH_SIZE,HIDDEN_SIZE])) 
    latent, mu, log_var = encoder(inputs)

    score = decoder(decoder_input,latent,lengths,SEQ_LENGTH)
    recon_loss=Recon(score,targets.view(-1))
    kld_loss = torch.sum(0.5 * (mu**2 + torch.exp(log_var) - log_var -1))
    #checker.append((recon_loss,kld_loss))
    
#     KL_COST_ANNEALING
    if recon_loss.data.numpy()[0]<1.5:
        KCA = 1.0

    else:
        KCA = 0.0
    ELBO = recon_loss+KCA*kld_loss
#     ELBO = recon_loss+kld_loss
    loss = ELBO.data.numpy()[0]
    
    ELBO.backward()
    
    
    
    torch.nn.utils.clip_grad_norm(encoder.parameters(), 5.0)
    torch.nn.utils.clip_grad_norm(decoder.parameters(), 5.0)
    
    dec_optim.step()
    enc_optim.step()
 
    
    if epoch % 100==0:
        #kindex+=1
        print("[%d/%d] ELBO : %.4f , RECON : %.4f & KLD : %.4f" % (epoch,STEP,ELBO.data.numpy()[0],
                                                                              recon_loss.data.numpy()[0],
                                                                              kld_loss.data.numpy()[0]))

[0/3000] ELBO : 1.5520 , RECON : 1.5520 & KLD : 1282.6074
[100/3000] ELBO : 1304.2341 , RECON : 1.4849 & KLD : 1302.7493
[200/3000] ELBO : 1.6225 , RECON : 1.6225 & KLD : 1243.2760
[300/3000] ELBO : 1.5753 , RECON : 1.5753 & KLD : 1258.8152
[400/3000] ELBO : 1.5726 , RECON : 1.5726 & KLD : 1247.9152
[500/3000] ELBO : 1291.0088 , RECON : 1.4193 & KLD : 1289.5895
[600/3000] ELBO : 1.5714 , RECON : 1.5714 & KLD : 1278.0747
[700/3000] ELBO : 1.5472 , RECON : 1.5472 & KLD : 1274.2760
[800/3000] ELBO : 1.5684 , RECON : 1.5684 & KLD : 1266.4414
[900/3000] ELBO : 1.6303 , RECON : 1.6303 & KLD : 1242.3254
[1000/3000] ELBO : 1.5573 , RECON : 1.5573 & KLD : 1265.3756
[1100/3000] ELBO : 1.5021 , RECON : 1.5021 & KLD : 1242.4119
[1200/3000] ELBO : 1.5088 , RECON : 1.5088 & KLD : 1276.9618
[1300/3000] ELBO : 1.5653 , RECON : 1.5653 & KLD : 1228.1581
[1400/3000] ELBO : 1.5243 , RECON : 1.5243 & KLD : 1235.4465
[1500/3000] ELBO : 1216.5645 , RECON : 1.3805 & KLD : 1215.1840
[1600/3000] ELBO : 1.6670 ,

## test 

### Recon

In [67]:
index=random.choice(range(300))
latent,_,_ = encoder(inputs[index].view(1,-1))
decoder_input = Variable(torch.LongTensor([[SOS_token]])).transpose(1,0)
#context = Variable(torch.randn([1,1,HIDDEN_SIZE])) 
recon = decoder(decoder_input,latent,lengths,SEQ_LENGTH,False)

v,i = torch.max(recon,1)

decoded=[]
for t in range(i.size()[0]):
    decoded.append(index2word[i.data.numpy()[t][0]])
    
print('Q: ', ' '.join([i for i in train[index][0] if i !='PAD' and i != 'EOS'])+'\n')
print('A: ', ' '.join([i for i in decoded if i !='PAD' and i != 'EOS'])+'\n')

Q:  결혼식 에 축화 화환 보낼려고 하 는데 시간 도 정하 면 맞출 수 있

A:  문 비서 축화 화환 보낼려고 하 는데 시간 도 정하 면 맞출 수 있



### Generate

In [195]:
decoder_input = Variable(torch.LongTensor([[SOS_token]])).transpose(1,0)
context = Variable(torch.randn([1,10])) 
recon = decoder(decoder_input,context,lengths,SEQ_LENGTH,False)

v,i = torch.max(recon,1)

decoded=[]
for t in range(i.size()[0]):
    decoded.append(index2word[i.data.numpy()[t][0]])

print('A: ', ' '.join([i for i in decoded if i !='PAD' and i != 'EOS'])+'\n')

A:  화환 꽃 부탁 요청



In [204]:
compare = [' '.join(tagger.morphs(d)) for d in data]

In [212]:
def generate(num):
    result=[]
    counter=0
    while counter<num:
        decoder_input = Variable(torch.LongTensor([[SOS_token]])).transpose(1,0)
        context = Variable(torch.randn([1,10])) 
        recon = decoder(decoder_input,context,lengths,SEQ_LENGTH,False)

        v,i = torch.max(recon,1)

        decoded=[]
        for t in range(i.size()[0]):
            dd = index2word[i.data.numpy()[t][0]]
            if dd not in decoded:
                decoded.append(dd)
        
        r = ' '.join([i for i in decoded if i !='PAD' and i != 'EOS'])
        
        if r not in compare:
            result.append(r)
            counter+=1
    return result

In [213]:
generated = generate(300)

In [214]:
len(generated)

300

In [217]:
with open('../../dataset/generated_FLOWER300.txt','w',encoding='utf-8') as f:
    for g in generated:
        f.write(g+'\n')

In [218]:
torch.save(encoder,'../../trained/flower_encoder.pkl')
torch.save(decoder, '../../trained/flower_generator.pkl')
# model = torch.load('model.pkl')

/usr/local/lib/python3.5/dist-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type EncoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.5/dist-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type DecoderRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
